## Mimic-III + ICD9 Data Analysis

**Challenges with AI based Medical Coding**  
  
This notebook will explore/demonstrate the data based challenges that make AI based medical coding difficult:  
- The Long Tail Problem
- Vast label space
- Medical Note Complexity

**Dataset Details**  
  
Data for this analysis is from the semi-open [MIMIC-III Clinical Dataset](https://physionet.org/content/mimiciii/1.4/). **The data is not permenantly stored in this repository.** To access the data, follow the steps on the linked website and complete the necessary trainings. This data is for research purposes only.

In [1]:
import plotly.express as px
import pandas as pd
import tiktoken
from src.icd9_tree import ICD9

pd.set_option('display.max_colwidth', None)

---
### Examine the Dataset

In [2]:
data_folder = "data/raw/"
d_icd = pd.read_csv(data_folder + 'D_ICD_DIAGNOSES.csv.gz', usecols=['ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']) # ICD Code lookup
diagnoses = pd.read_csv(data_folder + 'DIAGNOSES_ICD.csv.gz', usecols=['HADM_ID', 'ICD9_CODE']) # Linkage between ICD codes and Note events
drg = pd.read_csv(data_folder + 'DRGCODES.csv.gz', usecols=['HADM_ID','DESCRIPTION']) # DRG Codes
note_events= pd.read_csv(data_folder + 'NOTEEVENTS.csv.gz', usecols=['HADM_ID','TEXT'], nrows=20000) # Number of rows = 2,083,180 without filter # We will handle this later

In [3]:
# Init Code Tree
tree = ICD9('icd9_codes_full.json')

In [ ]:
# Data Sizes
print(f"D_ICD Shape: {d_icd.shape}")
print(f"DIAGNOSES Shape: {diagnoses.shape}")
print(f"DRG Shape: {drg.shape}")
print(f"Note Events Shape: {note_events.shape}")

In [ ]:
# Schemas
print(f"D_ICD Schema: {d_icd.columns}")
print(f"DIAGNOSES Schema: {diagnoses.columns}")
print(f"DRG Schema: {drg.columns}")
print(f"Note Events Schema: {note_events.columns}")

In [6]:
# Primary tables for this exercise are the DIAGNOSES and NOTE_EVENTS tables
# Joining these two tables will give us the linkage between the ICD codes and the notes

note_events['TEXT'] = note_events['TEXT'].apply(lambda x: "\"" + str(x) + "\"")
diagnoses = diagnoses.groupby(['HADM_ID']).agg(tuple).map(list).reset_index()
joined = note_events.join(diagnoses.set_index("HADM_ID"), on=['HADM_ID'], how='inner')

In [ ]:
print(joined.columns)
print(joined.shape)
display(joined.head(1))

---
### The Long Tail Problem
  
As with any classification problem, having a heavily skewed distribution presents a challenge with supervised learning. The MIMIC-III dataset presents this challenge to the MAX.

In [ ]:
# Get Distribution of codes from Diagnoses table
def remove_evm_codes(df: pd.DataFrame) -> pd.DataFrame:
    # Drop E, V, and M codes
    df['ICD9_CODE'] = df['ICD9_CODE'].apply(lambda x: str(x))
    output = df[~df['ICD9_CODE'].str.startswith("E")]
    output = output[~output['ICD9_CODE'].str.startswith("V")]   
    output = output[~output['ICD9_CODE'].str.startswith("M")]
    return output

# DIAGNOSES Schema: Index(['HADM_ID', 'ICD9_CODE'], dtype='object')
diagnoses = pd.read_csv(data_folder + 'DIAGNOSES_ICD.csv.gz', usecols=['HADM_ID', 'ICD9_CODE']) 
df = diagnoses[diagnoses['ICD9_CODE'].notna()]
df = df.drop(columns=['HADM_ID'])

# Drop codes that start with E, V, and M
df = remove_evm_codes(df)

# Shorten codes to 3 digits (for now)
df['ICD9_CODE'] = df['ICD9_CODE'].str.slice(0, 3)

# Explode list
df['ICD9_CODE'] = df['ICD9_CODE'].explode('ICD9_CODE').reset_index(drop=True)

# Get count for each ecode
df = df.value_counts().reset_index()
print(df.shape)


In [ ]:
# Plot ALL codes (for dramatic effect - this is not useful)

fig = px.bar(df.nlargest(columns=['count'], n=df.shape[0]), x='ICD9_CODE', y='count',color='ICD9_CODE', title='Code Counts')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()


In [ ]:
# Plot the top 200 codes - the long tail still extists even after taking a subset!!!

fig = px.bar(df.nlargest(columns=['count'], n=200), x='ICD9_CODE', y='count',color='ICD9_CODE', title='Code Counts')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
# The TOP X codes represent what percent of the MIMIC-III data?
x=int(input("Enter a number: "))

top_100 = df.nlargest(columns=['count'], n=x)
top_100_sum = top_100['count'].sum()
total_sum = df['count'].sum()

print(f"Top {x} Codes represent {top_100_sum/total_sum*100:.2f}% of the data")

---
### Vast Label Space  
  
For this exercise we omit 'E' and 'V' codes - however we are still left with a humungous label space. The quantity of potential labels presents challenges with multi-label classification - there are a lot of different options a model can choose from.

In [ ]:
# Read ICD9 codes in as a tree and view top level (we will call them L1 or Chapters) codes
# Note: L0 is the root node

# list of top level codes (e.g., '001-139', ...)
toplevelnodes = tree.children
for node in toplevelnodes:
    if node.code[0] not in ['E', 'V']:
        print(node.code, node.description)

In [ ]:
# Get counts of all code levels and plot results
L1 = 0
L2 = 0
L3 = 0
L4 = 0
L5 = 0

for node in tree.children:
    if node.code[0] not in ['E', 'V']:
        L1 += 1
        L2 += len(node.children)
        for l2_child in node.children:
            L3 += len(l2_child.children)
            for l3_child in l2_child.children:
                L4 += len(l3_child.children)
                for l4_child in l3_child.children:
                    L5 += len(l4_child.children)

print(f"Counts:\nL1: {L1}\nL2: {L2}\nL3: {L3}\nL4: {L4}\nL5: {L5}")

In [ ]:
# Plot the distribution of codes at each level
data = {'Level': ['L1', 'L2', 'L3', 'L4', 'L5'], 'Count': [L1, L2, L3, L4, L5]}
df = pd.DataFrame(data)
fig = px.bar(df, x='Level', y='Count', title='Code Counts by Level', color='Level')
fig.show()

---
### Note Complexity  
  
Note complexity presents another challenge to the classification problem. Note Texts from the Mimic dataset are...
- Poorly formatted
- Difficult to contextualize - for example the use of specific medical acronyms or sitations where a patient does NOT have xyz condition/symptom
- Lengthy (in terms of tokens)


In [ ]:
# Lets look at a ramdon note text
display(note_events.sample(1, random_state=123)[['TEXT']])

In [ ]:
# Get the average number of tokens in the notes

def get_token_counts(text):
    encoding = tiktoken.get_encoding('cl100k_base')
    num_tokens = len(encoding.encode(text))
    return num_tokens

note_events['num_tokens'] = note_events['TEXT'].apply(get_token_counts)
print(f"Average Token Count: {note_events['num_tokens'].mean()}")

---
### Plot L1 Code Distribution

In [ ]:
tree.find('191.1').parents[1].code

In [18]:
# Helper function to get L1 codes
def get_cat_code(code_list, level, tree):
    chapter_code_list = []
    error_list = []
    for code in code_list:
        try:
            if code[0] not in ['E', 'V']:
                if len(code) > 3:
                    code = code[:3] + '.' + code[3:]
                    chapter_code_list.append(tree.find(code).parents[level].code)
        except:
            if code not in error_list:
                error_list.append(code)
            
    # print(f"Error List: {error_list}")
    return list(set(chapter_code_list))

In [ ]:
# Get L1 codes
codes_df = joined.copy()
codes_df['L1'] = codes_df['ICD9_CODE'].apply(lambda x: get_cat_code(x, 1, tree))
display(codes_df[['L1', 'ICD9_CODE']].head(5))

In [21]:
# Get L1_Len
codes_df['L1_Len'] = codes_df['L1'].apply(lambda x: len(x))

In [ ]:
# plot L1_Len counts
fig = px.bar(codes_df.value_counts('L1_Len').reset_index(), x='L1_Len', y='count', title='L1 Length Counts')
fig.show()